In [1]:

import os

In [2]:

%pwd

'd:\\End to end machine learning projects\\Kidney Disease classification predication\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:

os.chdir("../")

In [4]:
%pwd

'd:\\End to end machine learning projects\\Kidney Disease classification predication\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URL"] = "https://dagshub.com/KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Kartik Nimhan"
os.environ["MLFLOW_TRACKING_PASSWORD"]="819c140b14906695425a85526c4e38f678bc82e3"

In [6]:
import dagshub
import mlflow

dagshub.init(repo_owner='KartikNimhan', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow")


Accessing as KartikNimhan

Initialized MLflow to track repo "KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project"

Repository KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project initialized!

In [7]:

import tensorflow as tf

In [8]:
import os

model_path = r"D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\training\model.h5"
if os.path.exists(model_path):
    print("Path exists.")
else:
    print("Path does not exist.")


Path exists.


In [9]:
import tensorflow as tf

model_path = r"D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\training\model.h5"
model = tf.keras.models.load_model(model_path)


In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [11]:

from cnnClassifier.constants import *
from cnnClassifier.utils.commom import read_yaml, create_directories, save_json

CONFIG_FILE_PATH: D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\config\config.yaml
PARAMS_FILE_PATH: D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml


In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="D:/End to end machine learning projects/Kidney Disease classification predication/Kidney-Disease-Classification-Deep-Learning-Project/artifacts/training/model.h5",
            training_data="D:/End to end machine learning projects/Kidney Disease classification predication/Kidney-Disease-Classification-Deep-Learning-Project/artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [14]:
import time
import mlflow
from urllib3.exceptions import ProtocolError
from mlflow.exceptions import MlflowException

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self, retries=3, delay=5):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        attempt = 0
        while attempt < retries:
            try:
                with mlflow.start_run():
                    mlflow.log_params(self.config.all_params)
                    mlflow.log_metrics(
                        {"loss": self.score[0], "accuracy": self.score[1]}
                    )
                    if tracking_url_type_store != "file":
                        mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                    else:
                        mlflow.keras.log_model(self.model, "model")
                break
            except (ProtocolError, MlflowException) as e:
                attempt += 1
                if attempt < retries:
                    print(f"Retry {attempt}/{retries} after {delay} seconds due to: {e}")
                    time.sleep(delay)
                else:
                    raise e


In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-02 18:04:25,015: INFO: commom: yaml file: D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\config\config.yaml loaded successfully]
[2024-08-02 18:04:25,023: INFO: commom: yaml file: D:\End to end machine learning projects\Kidney Disease classification predication\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml loaded successfully]
[2024-08-02 18:04:25,025: INFO: commom: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 55s 6s/step - loss: 9.6047 - accuracy: 0.6187
[2024-08-02 18:05:20,979: INFO: commom: json file saved at: scores.json]


2024/08/02 18:05:23 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-02 18:05:25,122: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Kartik\AppData\Local\Temp\tmpmspjl33l\model\data\model\assets
[2024-08-02 18:05:26,081: INFO: builder_impl: Assets written to: C:\Users\Kartik\AppData\Local\Temp\tmpmspjl33l\model\data\model\assets]
[2024-08-02 18:05:39,515: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/api/2.0/mlflow/runs/get?run_uuid=6dd79bbad97540d59192b09570b040a0&run

c:\Users\Kartik\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2024-08-02 18:05:43,684: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/api/2.0/mlflow/runs/get?run_uuid=6dd79bbad97540d59192b09570b040a0&run_id=6dd79bbad97540d59192b09570b040a0]
[2024-08-02 18:06:07,597: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /KartikNimhan/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/bd8d61a822b94896b51c2e9610a3305d/6dd79bbad97540d59192b09570b040a0/artifacts/model/data/model/variables/variables.dat

Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/08/02 18:07:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.
